In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import string
from tqdm import tqdm
import pandas as pd

service = ChromeService(executable_path="D:\Web_Driver\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.maximize_window()

In [2]:
driver.get('https://www.canadapharmacy.com/prescription')
time.sleep(1)

In [3]:
test_list = []
test_list = list(string.ascii_lowercase)

In [4]:
test_list_new = test_list[0:5]

In [ ]:
medicine_url_all = []
for i in tqdm(test_list):
    driver.get(f'https://www.canadapharmacy.com/prescription/{i}')
    for k in driver.find_elements(By.XPATH,"//h2[@class='mn']//a"):
        medicine_url_all2.append(k.get_attribute("href"))

In [9]:
len(medicine_url_all)

536

In [ ]:
data = []
sep = '\n'
for i in tqdm(medicine_url_all):
    st_brand = []
    st_generic = []
    tab_price_brand = []
    tab_price_generic = []
    driver.get(i)
    time.sleep(1)
    
    r = driver.page_source
    soup = BeautifulSoup(r,'lxml')
    
    card = ''.join([x.get_text(' ',strip=True) for x in soup.select('div.answer.expanded')])
    
    try:
        des = card.split('Directions')[0].replace('Description','')
    except:
        des = None
    
    try:
        drc = card.split('Directions')[1].split('Ingredients')[0]
    except:
        drc = None
        
    try:
        ingre= card.split('Directions')[1].split('Ingredients')[1].split('Cautions')[0]
    except:
        ingre = None
    
    try:
        cau = card.split('Directions')[1].split('Ingredients')[1].split('Cautions')[1].split('Side Effects')[0]
    except:
        cau = None
    try:
        se = card.split('Directions')[1].split('Ingredients')[1].split('Cautions')[1].split('Side Effects')[1]
    except: 
        se = None
        
    try:
        main_name = (soup.find('h1',{"class":"mn"}).text.lstrip()).rstrip()
    except:
        main_name = None
        
    try:
        pres = (soup.find('div',{"class":"card product brand strength equal"}).find('p').text.lstrip()).rstrip()
    except:
        pres = None
    
    try:
        pres2 = (soup.find('div',{"class":"card product generic strength equal"}).find('div').find('p').text.lstrip()).rstrip()
    except:
        pres2 = None
        
    try:
        brand_or_gen = (soup.find('div',{"class":"badge-container"}).find('div').text.lstrip()).rstrip()
    except:
        brand_or_gen = None 
        
    try:
        sec_name = ((soup.find('div',{"class":"card product brand strength equal"}).find('h3').text.lstrip()).rstrip()).split(sep, 1)[0]
    except:
        sec_name = None

    try:
        generic_name = (soup.find('div',{"class":"card product generic strength equal"}).find('div').find('h3').text.lstrip()).rstrip().split(sep, 1)[0]
    except:
        generic_name = None
        
    try:
        img_url_generic = driver.find_element(By.XPATH,"//img[@class='image generic']").get_attribute("src")
    except:
        img_url_generic = None
        
    try:
        img_url = driver.find_element(By.XPATH,"//img[@class='image brand']").get_attribute("src")
    except:
        img_url = None
        
    try:
        for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][1]"):
            st_brand.append(i.text)
    except:
        st_brand.append(None)
        
    try:
        for i in driver.find_elements(By.XPATH,"//div[@class='card product generic strength equal']//div//form//div//div[@class='product-select-options'][1]"):
            st_generic.append(i.text)
    except:
        st_generic.append(None)
        
    try:
        for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][1]"):
            tab_price_brand.append(i.text.replace('Select A Size...','').rstrip().replace('\n\n',' ').replace('\n',','))
    except:
        tab_price_brand.append(None)
    
    try:
        for i in driver.find_elements(By.XPATH,"//div[@class='card product generic strength equal']//div[@class='product-select-options-row']"):
            tab_price_generic.append(i.text.replace('Select A Size...','').rstrip().replace('\n\n',' ').replace('\n',','))
    except:
        tab_price_generic.append(None)
        
    for j in soup.find('div',{"class":"answer expanded"}).find_all('h4'):
        if 'Product Code' in j.text:
            prod_code = j.text
        
    data.append({
        'Name':main_name,
        'Medicine_Brand_Name':sec_name,
        'Medicine_Generic_Name': generic_name,
        'Prescription_Required_For_Medicine':pres,
        'Prescription_Required_for_Generic':pres2,
        'Product_Code':prod_code,
        'Product_Strength_Brand':st_brand,
        'Product_Price_Brand':tab_price_brand,
        'Product_Strength_Generic':st_generic,
        'Product_Price_Generic':tab_price_generic,
        'Product_Description':des,
        'Product_Directions':drc,
        'Prouct_Ingredients':ingre,
        'Product_Cautions':cau,
        'Product_Side_Effects':se,
        'Image_URL':img_url,
        'Image_URL Generic':img_url_generic})

  1%|▏         | 7/536 [00:33<45:59,  5.22s/it]

In [8]:
df = pd.DataFrame(data)

In [9]:
df.head()

,Name,Medicine_Brand_Name,Medicine_Generic_Name,Prescription_Required_For_Medicine,Prescription_Required_for_Generic,Product_Code,Product_Strength_Brand,Product_Price_Brand,Product_Strength_Generic,Product_Price_Generic,Product_Description,Product_Directions,Prouct_Ingredients,Product_Cautions,Product_Side_Effects,Image_URL,Image_URL Generic
0,Abilify (Aripiprazole),Abilify (Aripiprazole),Generic Equivalent - Abilify (Aripiprazole),Prescription Required,Prescription Required,Product Code : 5513,"[2mg, 5mg, 10mg, 15mg, 20mg, 30mg, 1mg/ml Solu...","[2mg, 5mg, 10mg, 15mg, 20mg, 30mg, 1mg/ml Solu...","[2mg, 5mg, 10mg, 15mg, 20mg, 30mg]","[2mg 100 tablets - $98.99, 5mg 100 tablets - $...",Abilify Tablet (Aripiprazole) Abilify (Aripip...,"Once you are prescribed and buy Abilify, then...",The active ingredient in Abilify medication i...,Abilify and other antipsychotic medications h...,The side effects of Abilify vary greatly depe...,https://www.canadapharmacy.com/content/images/...,https://www.canadapharmacy.com/content/images/...
1,Abilify Maintena (Aripiprazole),Abilify Maintena (Aripiprazole),None,Prescription Required,None,Product Code : 13246,[400mg Maintena],[400mg Maintena],[],[],ABILIFY is an antipsychotic medication used t...,Take ABILIFY as exactly prescribed by your do...,Aripiprazole,Some medical conditions may interact with ABI...,Patients should not use ABILIFY if they are a...,https://www.canadapharmacy.com/content/images/...,None
2,Abilify ODT (Aripiprazole),Abilify ODT (Aripiprazole),Generic Equivalent - Abilify ODT (Aripiprazole),Prescription Required,Prescription Required,Product Code : 11673,"[10mg ODT, 15mg ODT]","[10mg ODT, 15mg ODT]","[5mg ODT, 20mg ODT, 30mg ODT]","[5mg ODT 90 tablets - $59.00, 20mg ODT 90 tabl...",Abilify ODT is an antipsychotic medication us...,Place Abilify ODT directly on the tongue with...,The active ingredient in Abilify ODT is aripi...,Tell your doctor if you have a history of any...,Common Abilify side effects may include: Akat...,https://www.canadapharmacy.com/content/images/...,https://www.canadapharmacy.com/content/images/...
3,None,None,None,None,None,Product Code : 13701,[],[],[],[],House dust mite allergen extract belongs to t...,The recommended adult dose of this medication...,"Each white to off-white, circular sublingual ...",,None,None,None
4,Accolate (Zafirlukast),None,Generic Equivalent - Accolate (Zafirlukast),None,Prescription Required,Product Code : 1307,[],[],"[10mg, 20mg]","[10mg 60 tablets - $109.00, 20mg 60 tablets - ...","Accolate, 20 mg zafirlukast Dosage and Purpos...",The drug needs to be ingested by mouth (orall...,"The most active substance is Zafirlukast, wit...",Take all medicines with a doctor’s prescripti...,Although the side effects listed below are no...,None,https://www.canadapharmacy.com/content/images/...


In [10]:
df['Product_Price_Generic'][0]

['2mg 100 tablets - $98.99',
 '5mg 100 tablets - $43.99',
 '10mg 90 tablets - $38.59',
 '15mg 90 tablets - $56.59',
 '20mg 90 tablets - $67.99',
 '30mg 84 tablets - $75.99']

In [33]:
df.to_csv('D:\Medical_Scrape_project\data_final_1_12.csv')

In [2]:
driver.get('https://www.canadapharmacy.com/products/abilify-tablet')

In [3]:
for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][1]"):
    print(i.text)

2mg
5mg
10mg
15mg
20mg
30mg
1mg/ml Solution


In [29]:
test_med = []
for i in driver.find_elements(By.XPATH,"//div[@class='card product generic strength equal']//div//form//div//div[@class='product-select-options'][2]//div//select//option"):
    if i.text != 'Select A Size...':
        test_med.append(i.text)
    else:
        pass

In [30]:
test_med

['100 tablets - $98.99',
 '100 tablets - $43.99',
 '90 tablets - $38.59',
 '90 tablets - $56.59',
 '90 tablets - $67.99',
 '84 tablets - $75.99']

In [25]:
test_st_price = []
for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div[@class='product-select-options-row']"):
    test_st_price.append(i.text.replace('Select A Size...','').rstrip().replace('\n\n',' ').replace('\n',' , '))

In [26]:
test_st_price

['2mg 30 tablets - $219.99 , 90 tablets - $526.99',
 '5mg 28 tablets - $160.99 , 84 tablets - $459.99',
 '10mg 28 tablets - $116.99 , 84 tablets - $162.99',
 '15mg 28 tablets - $159.99 , 84 tablets - $198.99',
 '20mg 90 tablets - $745.99',
 '30mg 28 tablets - $104.99 , 84 tablets - $289.99',
 '1mg/ml Solution 150 ml - $239.99']

In [21]:
import requests

In [22]:
r = requests.get('https://www.canadapharmacy.com/products/abilify-tablet')

In [23]:
soup = BeautifulSoup(r.text, 'lxml')

In [27]:
sep = '\n'

In [28]:
((soup.find('div',{"class":"card product brand strength equal"}).find('h3').text.lstrip()).rstrip()).split(sep, 1)[0]

'Abilify (Aripiprazole)'

In [29]:
(soup.find('div',{"class":"card product generic strength equal"}).find('div').find('h3').text.lstrip()).rstrip().split(sep, 1)[0]

'Generic Equivalent - Abilify (Aripiprazole)'

In [157]:
df.to_csv('D:\Medical_Scrape_project\data_final_1_12.csv')

In [7]:
from datetime import date

In [8]:
today = date.today()
today

datetime.date(2022, 12, 1)

In [116]:
for i in range(len(df['Product Strength'][0])):
    print(df['Product Strength'][0][i])

2mg
5mg
10mg
15mg
20mg
30mg
1mg/ml Solution


In [112]:
for i in range(len(df['tab_price'])):
    df['Price'][i] = pd.Series(df['tab_price'][i])

C:\Users\DELL\AppData\Local\Temp\ipykernel_15216\1764651990.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Price'][i] = pd.Series(df['tab_price'][i])


In [115]:
df['Price'][24]

0               1 inhaler - $93.993 inhalers - $188.99
1    1 inhaler - $109.992 inhalers - $214.993 inhal...
2    1 inhaler - $99.992 inhalers - $199.003 inhale...
dtype: object

In [147]:
for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][1]"):
    print(i.text)

2mg
5mg
10mg
15mg
20mg
30mg
1mg/ml Solution


In [148]:
for i in driver.find_elements(By.XPATH,"//div[@class='card product generic strength equal']//div//form//div//div[@class='product-select-options'][1]"):
    print(i.text)

2mg
5mg
10mg
15mg
20mg
30mg


In [106]:
for i in range(len(df['Product Strength'])):
    df['Product Strength'][i].unique()

AttributeError: 'list' object has no attribute 'unique'

In [90]:
df['tab_price'][24][0]

'1 inhaler - $93.993 inhalers - $188.99'

In [ ]:
df[]

In [144]:
driver.get('https://www.canadapharmacy.com/products/abilify-tablet')

In [145]:
for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][2]//div//select//option[2]"):
    print(i.text)

30 tablets - $219.99
28 tablets - $160.99
28 tablets - $116.99
28 tablets - $159.99
90 tablets - $745.99
28 tablets - $104.99
150 ml - $239.99


In [143]:
for i in driver.find_elements(By.XPATH,"//div[@class='card product generic strength equal']//div//form//div//div[@class='product-select-options'][2]//div//select//option[2]"):
    print(i.text)

100 tablets - $98.99
100 tablets - $43.99
90 tablets - $38.59
90 tablets - $56.59
90 tablets - $67.99
84 tablets - $75.99


In [56]:
import requests

In [92]:
r = requests.get('https://www.canadapharmacy.com/products/lamictal')

In [93]:
soup = BeautifulSoup(r.text,'lxml')

In [94]:
try:
    print(((soup.find('div',{"class":"card product brand strength equal"}).find('h3').text.lstrip()).rstrip()).split(sep, 1)[0])
except:
    print("error")

Lamictal (Lamotrigine)


In [95]:
(soup.find('div',{"class":"card product generic strength equal"}).find('div').find('h3').text.lstrip()).rstrip().split(sep, 1)[0]

'Generic Equivalent - Lamictal (Lamotrigine)'

In [96]:
tab_price_test = []
for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][2]"):
    tab_price_test.append((((i.text).replace('Select A Size...','').replace('\n',' ')).lstrip()).rstrip())

In [102]:
tab_price_test[1]

'30 chewables - $29.99'

In [98]:
sth_test = []
for i in driver.find_elements(By.XPATH,"//div[@class='card product brand strength equal']//div//form//div//div[@class='product-select-options'][1]"):
    sth_test.append(i.text)

In [103]:
sth_test[1]

'5mg (Dispersible chew)'

In [47]:
(tab_price_test[0].replace('Select A Size...','')).replace('\n','')

'1 inhaler - $93.993 inhalers - $188.99'

In [32]:
import re

In [40]:
text = str(tab_price_test[0])
stri = ''
re.sub(r'^.*?n', text)

TypeError: sub() missing 1 required positional argument: 'string'

In [38]:
text

'Select A Size...\n1 inhaler - $93.99\n3 inhalers - $188.99\nSelect A Size...'

In [44]:
regex = r"^.*?n"
test_str = "Select A Size...\\n1 inhaler - $93.99\\n3 inhalers - $188.99\\nSelect A Size..."
subst = ""
result = re.sub(regex, subst, test_str)

In [45]:
result

'1 inhaler - $93.99\\n3 inhalers - $188.99\\nSelect A Size...'